# Gradient Asymmetry Stopping Experiment

**Complete Experiment Suite - All 16 Learning Rate Combinations**

**Learning Rate Combinations:**
- Actor LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Critic LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Total: **16 experiments**

---

## Features:
- Auto-saves to Google Drive every 100 episodes
- Auto-restores from Drive on session restart
- Crash recovery built-in
- Early stopping when all actors stop updating

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

## Cell 2: Setup - Choose Start Mode

The experiment script **automatically restores from Google Drive** when you run it. Choose how to proceed:

- **Option 1: Continue** - Run experiments (auto-restores completed ones from Drive)
- **Option 2: Start Fresh** - Clear ALL results (local + Drive) and start from scratch  
- **Option 3: Import from Zip** - Import results from a downloaded zip file

In [ ]:
import os
import zipfile
import shutil
import json

EXPERIMENT_NAME = 'stopping_experiment'
EXPERIMENT_ZIP = 'colab_experiment.zip'
RESULTS_DIR = '/content/results/stopping_experiment'
EXPERIMENT_DIR = '/content/stopping_experiment_run'
DRIVE_BACKUP_DIR = '/content/drive/MyDrive/gradient_asymmetry_results'

# ============================================================
# STEP 1: Extract experiment files
# ============================================================
if not os.path.exists(f'{EXPERIMENT_DIR}/mec_env.py'):
    drive_zip = f'/content/drive/MyDrive/{EXPERIMENT_ZIP}'
    
    if os.path.exists(drive_zip):
        print(f"Found experiment zip in Drive: {drive_zip}")
        zip_path = drive_zip
    else:
        print(f"Upload {EXPERIMENT_ZIP}:")
        from google.colab import files
        uploaded = files.upload()
        zip_path = list(uploaded.keys())[0]
    
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('/content')
    print("Experiment files extracted!")
else:
    print("Experiment files already present!")

# ============================================================
# STEP 2: Check existing progress
# ============================================================
drive_status_file = os.path.join(DRIVE_BACKUP_DIR, 'experiment_status.json')
if os.path.exists(drive_status_file):
    with open(drive_status_file) as f:
        drive_status = json.load(f)
    drive_completed = len(drive_status.get('completed', []))
    in_progress = drive_status.get('in_progress')
    print(f"\n*** Found Drive backup: {drive_completed}/16 completed ***")
    if in_progress:
        print(f"    Last in progress: {in_progress} (will restart this one)")
else:
    drive_completed = 0
    print("\n*** No existing Drive backup found ***")

# ============================================================
# CHOOSE START MODE
# ============================================================
print("\n" + "=" * 60)
print("SELECT START MODE")
print("=" * 60)
print("1. Continue - Auto-restore from Drive and run remaining")
print("2. Start Fresh - Clear ALL results and start from scratch")
print("3. Import from Zip - Upload results zip to restore")
print("=" * 60)

start_mode = input("Enter choice (1/2/3): ").strip()

if start_mode == '2':
    print("\n" + "=" * 60)
    print("STARTING FRESH - Clearing all results")
    print("=" * 60)
    if os.path.exists(RESULTS_DIR):
        shutil.rmtree(RESULTS_DIR)
        print(f"Cleared: {RESULTS_DIR}")
    if os.path.exists(DRIVE_BACKUP_DIR):
        shutil.rmtree(DRIVE_BACKUP_DIR)
        print(f"Cleared: {DRIVE_BACKUP_DIR}")
    print("Ready to run all 16 experiments from scratch!")

elif start_mode == '3':
    print("\n" + "=" * 60)
    print("IMPORT FROM ZIP")
    print("=" * 60)
    
    print("Upload your results zip file:")
    from google.colab import files
    uploaded = files.upload()
    results_zip_path = list(uploaded.keys())[0]
    
    temp_dir = '/content/temp_import'
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    with zipfile.ZipFile(results_zip_path, 'r') as z:
        z.extractall(temp_dir)
    
    actual_results = temp_dir
    for root, dirs, files_list in os.walk(temp_dir):
        if 'experiment_status.json' in files_list:
            actual_results = root
            break
    
    if os.path.exists(DRIVE_BACKUP_DIR):
        shutil.rmtree(DRIVE_BACKUP_DIR)
    shutil.copytree(actual_results, DRIVE_BACKUP_DIR)
    
    with open(os.path.join(DRIVE_BACKUP_DIR, 'experiment_status.json')) as f:
        imported_status = json.load(f)
    print(f"Imported {len(imported_status.get('completed', []))} completed experiments to Drive")
    print("The experiment script will auto-restore these when you run it.")
    
    shutil.rmtree(temp_dir)

else:
    print("\n" + "=" * 60)
    print("CONTINUE MODE")
    print("=" * 60)
    if drive_completed > 0:
        print(f"Will auto-restore {drive_completed} completed experiments from Drive")
    print("Ready to run!")

print("\n" + "=" * 60)
print("SETUP COMPLETE - Run the next cell to start experiments")
print("=" * 60)

## Cell 3: Check GPU and Environment

In [ ]:
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("")
    print("WARNING: No GPU detected!")
    print("Go to: Runtime -> Change runtime type -> GPU")

# Set working directory
os.chdir('/content')
print(f"\nWorking directory: {os.getcwd()}")

## Cell 4: Run All 16 Experiments

This will run all 16 experiments with:
- Actor LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Critic LRs: `[0.0001, 0.001, 0.01, 0.1]`

**Progress is auto-saved to Google Drive every 100 episodes.**

In [ ]:
import sys
sys.path.insert(0, '/content')
sys.path.insert(0, '/content/stopping_experiment_run')

from run_stopping_experiment_colab import run_all_experiments

# Run all experiments
run_all_experiments()

## Cell 5: Check Experiment Status

In [ ]:
import json
import os

results_dir = '/content/results/stopping_experiment'
status_file = os.path.join(results_dir, 'experiment_status.json')

if os.path.exists(status_file):
    with open(status_file) as f:
        status = json.load(f)
    print("Experiment Status:")
    print(f"  Completed: {len(status['completed'])}/16")
    if status['in_progress']:
        print(f"  In progress: {status['in_progress']}")
    print("\nCompleted experiments:")
    for exp in sorted(status['completed']):
        print(f"  - {exp}")
else:
    print("No status file found yet.")

# Also check Drive backup
drive_dir = '/content/drive/MyDrive/gradient_asymmetry_results'
if os.path.exists(drive_dir):
    print(f"\nDrive backup exists: {drive_dir}")
    contents = os.listdir(drive_dir)
    print(f"  Contents: {contents[:5]}..." if len(contents) > 5 else f"  Contents: {contents}")

## Cell 6: View Results Summary

In [ ]:
import json
import os

results_dir = '/content/results/stopping_experiment'

if os.path.exists(results_dir):
    print("Experiment Results Summary:")
    print("="*70)
    print(f"{'Actor LR':<12} {'Critic LR':<12} {'Stop Ep.':<12} {'Final Reward':<15}")
    print("-"*70)
    
    for exp_dir in sorted(os.listdir(results_dir)):
        result_file = os.path.join(results_dir, exp_dir, 'results.json')
        if os.path.exists(result_file):
            with open(result_file) as f:
                data = json.load(f)
            print(f"{data['actor_lr']:<12} {data['critic_lr']:<12} {data['stopping_episode']:<12} {data['final_reward']:<15.4f}")
    print("="*70)
else:
    print("No results directory found yet.")

## Cell 7: Download Results (Optional)

Results are auto-backed up to Drive, but you can also download a zip.

In [ ]:
import shutil
import os

results_dir = '/content/results'
output_zip = '/content/experiment_results.zip'

if os.path.exists(results_dir):
    shutil.make_archive('/content/experiment_results', 'zip', results_dir)
    print(f"Created: {output_zip}")
    print(f"Size: {os.path.getsize(output_zip) / 1024:.1f} KB")
    
    # Download
    from google.colab import files
    files.download(output_zip)
else:
    print("No results directory found.")